In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.tree import DecisionTreeRegressor

import pandas as pd
df_org = pd.read_csv("./Datasets/card_cust.csv")
df_org.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0
2,10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0.0,12.0,7500.0,622.066742,627.284787,0.000000,12.0
3,10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1.0,1.0,7500.0,0.000000,NaN,0.000000,12.0
4,10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0.0,1.0,1200.0,678.334763,244.791237,0.000000,12.0


In [14]:
df_org.info

<bound method DataFrame.info of      CUST_ID      BALANCE  BALANCE_FREQUENCY  PURCHASES  ONEOFF_PURCHASES  \
0      10001    40.900749           0.818182      95.40              0.00   
1      10002  3202.467416           0.909091       0.00              0.00   
2      10003  2495.148862           1.000000     773.17            773.17   
3      10004  1666.670542           0.636364    1499.00           1499.00   
4      10005   817.714335           1.000000      16.00             16.00   
..       ...          ...                ...        ...               ...   
995    11029  1250.394614           0.909091     443.99            443.99   
996    11030     9.503968           1.000000      96.62              0.00   
997    11031  2285.068731           1.000000       0.00              0.00   
998    11032  2928.756699           1.000000     160.92              0.00   
999    11033  1460.178578           1.000000     937.96            100.00   

     INSTALLMENTS_PURCHASES  CASH_ADVANCE  

In [3]:
df_org.isna().sum()

CUST_ID                              0
BALANCE                              0
BALANCE_FREQUENCY                    0
PURCHASES                            0
ONEOFF_PURCHASES                     0
INSTALLMENTS_PURCHASES               0
CASH_ADVANCE                         0
PURCHASES_FREQUENCY                  0
ONEOFF_PURCHASES_FREQUENCY           0
PURCHASES_INSTALLMENTS_FREQUENCY     0
CASH_ADVANCE_FREQUENCY               0
CASH_ADVANCE_TRX                     0
PURCHASES_TRX                        0
CREDIT_LIMIT                         0
PAYMENTS                             0
MINIMUM_PAYMENTS                    74
PRC_FULL_PAYMENT                     0
TENURE                               0
dtype: int64

In [6]:
df1=df_org.copy()
df1['MINIMUM_PAYMENTS']=df1['MINIMUM_PAYMENTS'].fillna(df1['MINIMUM_PAYMENTS'].mean())
df_base=df1.copy()

# Groupby 결과에 unstack 적용

In [21]:
df1=df1[['TENURE','BALANCE','CREDIT_LIMIT']].copy()
round(df1.groupby('TENURE')[['BALANCE','CREDIT_LIMIT']].corr().unstack()['BALANCE']['CREDIT_LIMIT'].max(),2)


0.95

In [31]:
df2=df_base.copy()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CUST_ID                           1000 non-null   int64  
 1   BALANCE                           1000 non-null   float64
 2   BALANCE_FREQUENCY                 1000 non-null   float64
 3   PURCHASES                         1000 non-null   float64
 4   ONEOFF_PURCHASES                  1000 non-null   float64
 5   INSTALLMENTS_PURCHASES            1000 non-null   float64
 6   CASH_ADVANCE                      1000 non-null   float64
 7   PURCHASES_FREQUENCY               1000 non-null   float64
 8   ONEOFF_PURCHASES_FREQUENCY        1000 non-null   float64
 9   PURCHASES_INSTALLMENTS_FREQUENCY  1000 non-null   float64
 10  CASH_ADVANCE_FREQUENCY            1000 non-null   float64
 11  CASH_ADVANCE_TRX                  1000 non-null   float64
 12  PURCHAS

In [32]:
df2=df2.drop(columns='CUST_ID')
scaler=StandardScaler()
df2_scaler = scaler.fit_transform(df2)

[2, 3, 4, 5]

In [35]:
k_value=[*range(2,6)]
score=[]
for k in k_value:
    kmeans=KMeans(n_clusters=k,random_state=1234)
    kmeans.fit(df2_scaler)
    s=silhouette_score(df2_scaler,kmeans.labels_)
    score=score+[s]
score_series=pd.Series(score,index=k_value)
max_k=score_series.idxmax()


In [36]:
kmeans=KMeans(n_clusters=max_k,random_state=1234)
kmeans.fit(df2_scaler)

KMeans(n_clusters=2, random_state=1234)

In [38]:
df2['cluster']=kmeans.labels_
df2['cluster'].unique()

array([0, 1])

In [40]:
round(df2.groupby('cluster')['ONEOFF_PURCHASES'].mean().max(),2)

3946.19

In [42]:
seed = 1234
df = df_base.copy()

In [43]:
df_train = df.loc[df["CUST_ID"] % 4 != 0]
df_test  = df.loc[df["CUST_ID"] % 4 == 0]
len(df_train), len(df_test)

(752, 248)

In [45]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 752 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CUST_ID                           752 non-null    int64  
 1   BALANCE                           752 non-null    float64
 2   BALANCE_FREQUENCY                 752 non-null    float64
 3   PURCHASES                         752 non-null    float64
 4   ONEOFF_PURCHASES                  752 non-null    float64
 5   INSTALLMENTS_PURCHASES            752 non-null    float64
 6   CASH_ADVANCE                      752 non-null    float64
 7   PURCHASES_FREQUENCY               752 non-null    float64
 8   ONEOFF_PURCHASES_FREQUENCY        752 non-null    float64
 9   PURCHASES_INSTALLMENTS_FREQUENCY  752 non-null    float64
 10  CASH_ADVANCE_FREQUENCY            752 non-null    float64
 11  CASH_ADVANCE_TRX                  752 non-null    float64
 12  PURCHASE

In [46]:
y_train = df_train.pop('ONEOFF_PURCHASES')
y_test =  df_test.pop('ONEOFF_PURCHASES')

X_train=df_train.drop(columns='CUST_ID')
X_test=df_test.drop(columns='CUST_ID')

In [47]:
model_dtr = DecisionTreeRegressor(random_state = seed)
model_dtr.fit(X = X_train,y = y_train)

DecisionTreeRegressor(random_state=1234)

In [48]:
y_pred = model_dtr.predict(X_test)

In [49]:
round(((y_test - y_pred) ** 2).mean() ** 0.5, 1)

1039.2

In [44]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.tree import DecisionTreeRegressor

In [19]:
df1=df[['TENURE','BALANCE','CREDIT_LIMIT']].copy()
df1.head()

,TENURE,BALANCE,CREDIT_LIMIT
0,12.0,40.900749,1000.0
1,12.0,3202.467416,7000.0
2,12.0,2495.148862,7500.0
3,12.0,1666.670542,7500.0
4,12.0,817.714335,1200.0


In [102]:
df1.TENURE.nunique()
cor=df1.groupby('TENURE')[['BALANCE','CREDIT_LIMIT']].corr()
round(max(cor.unstack()['BALANCE']['CREDIT_LIMIT']),2)
# df1_cor=df1.corr()
# df1_cor

0.95

In [36]:
round(0.948405,2)

0.95

In [57]:
df2=df.copy()
df2=df2.loc[:,'BALANCE':]
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   BALANCE                           1000 non-null   float64
 1   BALANCE_FREQUENCY                 1000 non-null   float64
 2   PURCHASES                         1000 non-null   float64
 3   ONEOFF_PURCHASES                  1000 non-null   float64
 4   INSTALLMENTS_PURCHASES            1000 non-null   float64
 5   CASH_ADVANCE                      1000 non-null   float64
 6   PURCHASES_FREQUENCY               1000 non-null   float64
 7   ONEOFF_PURCHASES_FREQUENCY        1000 non-null   float64
 8   PURCHASES_INSTALLMENTS_FREQUENCY  1000 non-null   float64
 9   CASH_ADVANCE_FREQUENCY            1000 non-null   float64
 10  CASH_ADVANCE_TRX                  1000 non-null   float64
 11  PURCHASES_TRX                     1000 non-null   float64
 12  CREDIT_

In [107]:
scaler=StandardScaler()
scaler.fit(df2)
df2_s=scaler.fit_transform(df2)
df2_s

array([[-0.84876759, -0.41987944, -0.4419358 , ..., -0.46554357,
         0.28242902, -0.49687301],
       [ 0.28279099,  0.01213096, -0.4690169 , ...,  0.33159169,
         0.28242902, -0.49687301],
       [ 0.0296341 ,  0.44414137, -0.24953794, ..., -0.46554357,
         0.28242902, -0.49687301],
       ...,
       [-0.04555583,  0.44414137, -0.4690169 , ..., -0.46554357,
         0.28242902, -0.49687301],
       [ 0.18482699,  0.44414137, -0.4233367 , ..., -0.46554357,
         0.28242902, -0.49687301],
       [-0.34079285,  0.44414137, -0.20275918, ..., -0.46554357,
         0.28242902, -0.49687301]])

In [109]:
s=[]
for i in range(2,6):
    k_means=KMeans(n_clusters=i,random_state=1234)
    k_means.fit(df2_s)
#     labels = k_means.predict(df2_s)
    s=s+[silhouette_score(df2_s,k_means.labels_)]
print(s)
k_means=KMeans(n_clusters=2,random_state=1234)
k_means.fit(df2_s)
# labels = k_means.predict(df2_s)

df2['gr']=k_means.labels_
df2.head()
round(df2.groupby('gr')['ONEOFF_PURCHASES'].mean().max(),2)

[0.3595069758561101, 0.26093411474800915, 0.26625517192779835, 0.1914554864677741]


3946.19

In [92]:
df3=df.copy()
df3.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0
2,10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0.0,12.0,7500.0,622.066742,627.284787,0.000000,12.0
3,10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1.0,1.0,7500.0,0.000000,1297.116322,0.000000,12.0
4,10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0.0,1.0,1200.0,678.334763,244.791237,0.000000,12.0


In [104]:
df3_test=df3.loc[df3['CUST_ID']%4==0]
df3_y_test=df3_test['ONEOFF_PURCHASES']
df3_test=df3_test.drop(columns=['CUST_ID','ONEOFF_PURCHASES'],axis=1)
df3_train=df3.loc[df3['CUST_ID']%4!=0]
df3_y_train=df3_train['ONEOFF_PURCHASES']
df3_train=df3_train.drop(columns=['CUST_ID','ONEOFF_PURCHASES'],axis=1)

# df3_train.head()

In [105]:
print(df3_test.shape[0],df3_y_test.shape[0])
tree=DecisionTreeRegressor().fit(df3_train, df3_y_train)

248 248


tree=DecisionTreeRegressor().fit(df3_test, df3_y_test)

In [122]:
y_pre=tree.predict(df3_test)

In [123]:
df3_y_train.array

<PandasArray>
[    0.0,     0.0,  773.17,    16.0,     0.0, 6402.63,  661.49,  1281.6,
     0.0, 2500.23,
 ...
     0.0,  132.81,     0.0,  179.56,     0.0,  652.94,  443.99,     0.0,
     0.0,   100.0]
Length: 752, dtype: float64

In [124]:
y_pre-df3_y_train.array

ValueError: operands could not be broadcast together with shapes (248,) (752,) 